## Remote Graph from LangGraph Platform


In [ ]:
from langgraph.pregel.remote import RemoteGraph
from langgraph_sdk import get_sync_client
from langgraph.types import Command

url = "http://0.0.0.0:8000"
graph_name = "main_graph"
sync_client = get_sync_client(url=url)
remote_graph = RemoteGraph(graph_name, url=url)

# create a thread (or use an existing thread instead)
thread = sync_client.threads.create()

# obtain the graph with the thread config
config = {"configurable": {"thread_id": thread["thread_id"]}}

In [ ]:
# Define the input and invoke
user_input = "Ey, puedes resumirme 10 mensajes?"
message_input = {"messages": [{"role": "human", "content": user_input}]}
for event in remote_graph.stream(message_input, config, stream_mode="updates"):
    print(event)
    print("\n")


In [ ]:
# Continue the action
for event in remote_graph.stream(
    # provide continue
    Command(resume={"action": "continue"}),
    config,
    stream_mode="updates",
):
    print(event)
    print("\n")

## Local Graph Serverless

In [ ]:
## From LangGraph
from langgraph.types import Command
from langgraph.checkpoint.memory import MemorySaver
from cheese.workflow_builder import WorkflowBuilder
from cheese.config.config_graph import ConfigGraph
from cheese.entity.models.stategraph import SharedState
from cheese.entity.models.inputgraph import InputState
from cheese.entity.models.outputgraph import OutputState
from cheese.utils.common import read_yaml
from cheese.utils.logger import setup_logging
from cheese.constants import *

## Read the config.yaml
config = read_yaml(CONFIG_FILE_PATH)

## Setup logging Configuration
setup_logging(config)

## Workflow Configuration
workflow_builder = WorkflowBuilder(
    config=ConfigGraph, 
    state_schema=SharedState, 
    input=InputState, 
    output=OutputState,
    checkpointer=MemorySaver(),
    name=config['package']['repo_name']
)
cheese = workflow_builder.compile() # compile the graph
workflow_builder.display_graph(save=True, filepath="artifacts/cheese_graph.png") # update the graph artifact

In [2]:
# Define the thread_id
config = {"configurable": {"thread_id": "001"}}

In [ ]:
# Define the input and invoke
user_input = "Ey, ¿qué tal?"
message_input = {"messages": [{"role": "human", "content": user_input}]}
for event in cheese.stream(message_input, config, stream_mode="updates"):
    print(event)
    print("\n")

In [ ]:
# Define the input and invoke
user_input = "¿Puedes resumirme los últimos 25 mensajes?"
message_input = {"messages": [{"role": "human", "content": user_input}]}
for event in cheese.stream(message_input, config, stream_mode="updates"):
    print(event)
    print("\n")

In [ ]:
for event in cheese.stream(
    # provide feedback
    Command(resume={
        "action": "feedback", 
        "data": "Perdona, quise decir 5 mensajes"
        }
    ),
    config,
    stream_mode="updates",
):
    print(event)
    print("\n")

In [ ]:
for event in cheese.stream(
    # provide continue
    Command(resume={"action": "continue"}),
    config,
    stream_mode="updates",
):
    print(event)
    print("\n")